In [9]:
from django_pandas.io import read_frame

import os
os.environ['DJANGO_SETTINGS_MODULE'] = 'winesurvey.settings'
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

import django
django.setup()

from survey.models import WineItem

qs = WineItem.objects.all()

df = read_frame(qs)

df[df['vintage'].notna()]
df[df['closure_type'] == 'Screwtop']

,item_key,name,description,price,bottle_image_url,label_type,closure_type,country,region,origin,...,varietal,vintage,point_score,story,user_rating,sale_price,original_url,tasting_notes,winc_product_id,winc_product_code
22,winc10659,2013 Rambutan Gewürztraminer,Gewürztraminer is like the grown up version of...,16.99,https://d207gb2bfvg73.cloudfront.net/1410489/h...,None,Screwtop,United States,Sonoma,"Sonoma, California",...,Gewürztraminer,2013.0,None,,3.7,12.99,None,None,10659,1410489
64,winc12419,2018 Les Bobos Grenache,"Go bohemian with this bright, fun, food-friend...",15.99,https://d207gb2bfvg73.cloudfront.net/11191652/...,None,Screwtop,France,Vin de France,"Vin de France, France",...,Grenache,2018.0,None,Why so serious? Lighten things up with Les Bob...,3.9,13.99,None,None,12419,11191652
65,winc12400,2018 Undercard® Cabernet Sauvignon,Don’t underestimate the strength of this Paso ...,17.99,https://d207gb2bfvg73.cloudfront.net/08191635/...,None,Screwtop,United States,Paso Robles,"Paso Robles, California",...,Cabernet Sauvignon,2018.0,None,"Though she be but little, she is fierce. Napa ...",4.2,15.99,None,None,12400,08191635
71,winc12293,2018 Cape Route Chenin Blanc,From renowned California winemaker Ernst Storm...,14.99,https://d207gb2bfvg73.cloudfront.net/11181521/...,None,Screwtop,South Africa,Western Cape,"Western Cape, South Africa",...,Chenin Blanc,2018.0,None,"Cape Route is the creation of Ernst Storm, a w...",3.8,12.99,None,None,12293,11181521
73,winc12149,2015 Rogue Admirals Cabernet Sauvignon,This Australian Cabernet was crafted from sele...,14.99,https://d207gb2bfvg73.cloudfront.net/17101372/...,None,Screwtop,Australia,Limestone Coast,"Limestone Coast, Australia",...,Cabernet Sauvignon,2015.0,None,When Australian red wines really became popula...,3.9,12.99,None,None,12149,17101372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,winc12271,2017 TBT White Wine Blend,"This tropical, six-grape white blend is as pla...",14.99,https://d207gb2bfvg73.cloudfront.net/08171493/...,None,Screwtop,United States,,California,...,White Blend,2017.0,None,When friends come over for an impromptu Thursd...,3.5,12.99,None,None,12271,08171493
977,winc12113,2016 Wonderful Wine Co.® White Blend,"Lychee, white flower and exotic spice illumina...",14.99,https://d207gb2bfvg73.cloudfront.net/17071337/...,None,Screwtop,United States,,California,...,White Blend,2016.0,None,This wine is a state of mind — easy-as-it-gets...,3.8,12.99,None,None,12113,17071337
979,winc12042,2016 Summer Water® Rosé,Summer Water is more than the rosé of the seas...,18.99,https://d207gb2bfvg73.cloudfront.net/17031259/...,None,Screwtop,United States,Central Coast,"Central Coast, California",...,Rosé,2016.0,None,This dry rosé reflects the pale pink dream wor...,4.0,16.99,None,None,12042,17031259
980,winc12090,2016 Cocomero® Rosé,"A California rosé with Italian roots, this Bar...",14.99,https://d207gb2bfvg73.cloudfront.net/17071310/...,None,Screwtop,United States,,California,...,Rosé,2016.0,None,Inspired by the summer treat of choice in Ital...,3.8,12.99,None,None,12090,17071310
